Vamos tentar prever o custo do seguro médico

Colunas:

age (Idade): Idade do beneficiário principal

sex (sexo): contratante de seguros sexo, feminino, masculino

bmi (IMC): Índice de massa corporal, proporcionando uma compreensão do corpo, pesos relativamente altos ou baixos em relação à altura,
índice objetivo do peso corporal (kg/m^2) utilizando a relação altura/peso, idealmente 18,5 a 24,9

children (filhos): Número de filhos cobertos por plano de saúde / Número de dependentes

smoker (Fumante): Fumante

reginon (região): área residencial do beneficiário nos EUA, nordeste, sudeste, sudoeste, noroeste.

charges (Encargos): Custos médicos individuais faturados pelo seguro de saúde


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb


# <h3>REGRESSÃO LINEAR MÚLTIPLA</h3>

-> Carregando o conjunto de dados

In [2]:
df = pd.read_csv('insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
# Informações sobre os dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [4]:
# Resumo Estatístico
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [5]:
# Tem dados faltantes (NaN)?
df.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

-> Converter variáveis categóricas em numéricas

In [6]:
df = pd.get_dummies(df, columns=['sex', 'smoker', 'region'], drop_first=True)

In [7]:
df.head()

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,False,True,False,False,True
1,18,33.770,1,1725.55230,True,False,False,True,False
2,28,33.000,3,4449.46200,True,False,False,True,False
3,33,22.705,0,21984.47061,True,False,True,False,False
4,32,28.880,0,3866.85520,True,False,True,False,False


-> Variáveis independentes e dependente

In [8]:
X = df.drop('charges', axis=1)
y = df['charges']

-> Dados de treino e teste

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

-> Inicializando e treinando o modelo

In [10]:
reg_mult = LinearRegression()
reg_mult.fit(X_train, y_train)

LinearRegression()

-> Previsões

In [11]:
reg_mult_pred = reg_mult.predict(X_test)

-> Avaliação do modelo

In [12]:
mse = mean_squared_error(y_test, reg_mult_pred)
r2 = r2_score(y_test, reg_mult_pred)

print(f'Erro médio quadrático (MSE): {mse:.2f}')
print(f'R-squared (R²): {r2:.2f}')

Erro médio quadrático (MSE): 33596915.85
R-squared (R²): 0.78


O MSE é uma métrica que mede a média dos quadrados dos erros entre as previsões do modelo e os valores reais. Quanto menor o valor do MSE, melhor o ajuste do modelo aos dados. No nosso caso, um MSE de 33.596.915,85 indica que, em média, as previsões do modelo têm um erro quadrático médio de aproximadamente 33 milhões de unidades quadradas em relação aos valores reais do custo do seguro.

O MSE obtido é muito menor do que a variância dos valores reais de "charges" (que é aproximadamente 12110.01^2 = 146.553.139). Isso sugere que o modelo está fazendo previsões que, em média, estão relativamente próximas dos valores reais.


O R-squared de 0.78 indica que o modelo explica aproximadamente 78% da variabilidade nos custos do seguro com base nas características dos pacientes. Isso é um bom resultado.

-> Previsões com novos dados

In [13]:
novos_dados = pd.DataFrame({
    'age': [35],
    'bmi': [32.741],
    'children': [2],
    'sex_male': [1], # masculino
    'smoker_yes': [0], # não fumante
    'region_northwest': [1],
    'region_southeast': [0],
    'region_southwest': [0]
})

novas_previsões = reg_mult.predict(novos_dados)

previsao_valor = novas_previsões[0]

print(f'Custo previsto do seguro: {previsao_valor:.2f}')

Custo previsto do seguro: 8560.97


# <h3>REGRESSÃO RIDGE E LASSO</h3>

<h5>1. RIDGE</h5>

-> Criar o modelo

In [14]:
ridge_model = Ridge(alpha=1.0,solver='auto')

-> Treinar o modelo

In [15]:
ridge_model.fit(X_train, y_train)

Ridge()

-> Previsões

In [16]:
ridge_model_pred = ridge_model.predict(X_test)

-> Avaliação do modelo

In [17]:
mse_ridge = mean_squared_error(y_test, ridge_model_pred)
r2_ridge = r2_score(y_test, ridge_model_pred)

print(f'Regressão Ridge - Erro Médio Quadrático: {mse_ridge:.2f}')
print(f'Regressão Ridge - R-squared: {r2_ridge:.2f}')

Regressão Ridge - Erro Médio Quadrático: 33645037.09
Regressão Ridge - R-squared: 0.78


<h5>2. LASSO</h5>

-> Criar o modelo

In [18]:
lasso_model = Lasso(alpha=1)

-> Treinar o modelo

In [19]:
lasso_model.fit(X_train, y_train)

Lasso(alpha=1)

-> Previsões

In [20]:
lasso_model_pred = lasso_model.predict(X_test)

-> Avaliação

In [21]:
mse_lasso = mean_squared_error(y_test, lasso_model_pred)
r2_lasso = r2_score(y_test, lasso_model_pred)

print(f"Regressão Lasso - Erro Médio Quadrático: {mse_lasso:.2f}")
print(f"Regressão Lasso - R-squared: {r2_lasso:.2f}")

Regressão Lasso - Erro Médio Quadrático: 33605507.55
Regressão Lasso - R-squared: 0.78


# <h3>XGBoost</h3>

-> Criar o modelo

In [28]:
xgb_model = xgb.XGBRegressor(
    objective = 'reg:squarederror',
    n_estimators = 100,
    learning_rate = 0.1,
    max_depth = 3,
    random_state = 42
)

-> Treinar o modelo

In [29]:
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

-> Previsões

In [30]:
xgb_model_pred = xgb_model.predict(X_test)

-> Avaliação

In [31]:
# Avalie o modelo
mse = mean_squared_error(y_test,xgb_model_pred)
r2 = r2_score(y_test,xgb_model_pred)

# Exiba as métricas de avaliação
print(f'Erro médio quadrático (MSE): {mse:.2f}')
print(f'R-squared (R²): {r2:.2f}')

Erro médio quadrático (MSE): 18030522.49
R-squared (R²): 0.88


-> Previsões com novos dados

In [ ]:
# Relembrando os novos dados
'''novos_dados = pd.DataFrame({
    'age': [35],
    'bmi': [32.741],
    'children': [2],
    'sex_male': [1], # masculino
    'smoker_yes': [0], # não fumante
    'region_northwest': [1],
    'region_southeast': [0],
    'region_southwest': [0]
})'''

In [32]:
novas_previsões = xgb_model.predict(novos_dados)

previsao_valor = novas_previsões[0]

print(f'Custo previsto do seguro: {previsao_valor:.2f}')

Custo previsto do seguro: 8110.23


<h3>RESULTADO</h3>

Com base nos resultados do R²:

- Regressão Linear Múltipla: 0.78

- Regressão Ridge e Lasso: 0.78

- XGBoost: 0.88

Este último foi o que melhor desempenho teve para prever o custo de seguro.
